# 城市“西安”数据预处理

## 导入库&预设

In [190]:
ExtraDictionaryPath = 'set/dict.txt'
StopWordPath = 'set/stopwords.txt'
KeywordFilePath = 'resource/编码框架及关键词列表.xlsx'
KeywordSheetNameTopic = 'Sheet1'
KeywordSheetNameNarration = 'Sheet2'
KeywordTopNumber = 10
minLengthOfWord = 3

In [191]:
import sys
import os
import openpyxl
import csv
import re
import pandas as pd
import jieba
import jieba.posseg as psg
import matplotlib.pyplot as plt
import numpy as np
import cn2an
from loguru import logger
logger.remove()
handler_id = logger.add(sys.stderr, level="DEBUG")
# 支持中文
plt.rcParams['font.sans-serif'] = ['SimSong-Regular'] # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False # 用来正常显示负号
# 显示清晰
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## 处理工具

In [2]:
# 总体数据清洗
def clean_data(content):
    # 删去空格
    content = str(content).replace('\u200b', '').strip()
    return content

In [2]:
# 时间清洗
def clean_time_data(content):
    content = re.sub('[\u4e00-\u9fa5]', '', content).strip()
    content = pd.Timestamp(content)
    return content

In [193]:
# 内容清洗
def clean_content(content):
    content = re.sub('<[^<]+?>', '', str(content)
                     ).replace('\n', '').strip()  # 删除html文本
    content = re.sub(r'https://[a-zA-Z0-9.?/&=:]*',
                         '', str(content))  # 删除文本中的网址
    content = re.sub(r'http://[a-zA-Z0-9.?/&=:]*', '', content)
    return content

In [194]:
# 数字清洗
def clean_number(content):
    if (not content) or pd.isnull(content) or pd.isna(content):
        content = '0'
    content = str(content).replace(' ', '').replace('+', '')
    if not bool(re.search(r'\d', content)):
        content = '0'
    if bool(re.search(r'\D', content)):
        return cn2an.cn2an(content.replace(',','').strip(), 'smart')
    return content

In [195]:
#分词工具
def chinese_word_cut(my_text, min_length_of_word = minLengthOfWord, flag_list = ['n','nz','vn']):
    jieba.load_userdict(ExtraDictionaryPath)
    jieba.initialize()
    try:
        stopword_list_unprocessed = open(StopWordPath,encoding ='utf-8')
    except:
        stopword_list_unprocessed = []
        print("error in stop_file")
    stop_word_list = []
    for line in stopword_list_unprocessed:
        line = re.sub(u'\n|\\r', '', line)
        stop_word_list.append(line)

    #jieba分词
    word_list = []
    seg_list = psg.cut(my_text)
    for seg_word in seg_list:
        # word = re.sub(u'[^\u4e00-\u9fa5]','',seg_word.word)
        word = seg_word.word
        is_unfit = False
        for stop_word in stop_word_list:
            if stop_word == word or len(word) <  min_length_of_word:     #this word is stopword
                    is_unfit = True
                    break
        if (not is_unfit) and seg_word.flag in flag_list:
            word_list.append(word)      
    return (" ").join(word_list)

In [196]:
def data_preprocess(df_name, app_name):
    df_name.applymap(clean_data)
    df_name.to_pickle(f'data/{app_name}西安原始数据.pkl')
    

## 数据处理

In [197]:
weibo = pd.read_csv("resource/西安微博.csv")
# weibo = weibo.sample(10)
weibo.rename(columns={'微博正文':'内容', 'user_id':'user_tag'}, inplace = True)
weibo = weibo[['user_tag', '内容', '转发数', '评论数', '点赞数', '发布时间']]
weibo['平台'] = '微博'
print(weibo.columns.to_list())
display(weibo)

['user_tag', '内容', '转发数', '评论数', '点赞数', '发布时间', '平台']


/opt/anaconda3/envs/sklearn/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,user_tag,内容,转发数,评论数,点赞数,发布时间,平台
0,3454068732,#西安身边事#网友@心之所系家之所在_715:#西安新鲜事#求转发！！！陕西省西安市经开区2...,20289,182,7159,2017-08-06 13:12,微博
1,5283926940,#西安身边事#西安钟楼旁，@西安音乐学院同学公益快闪表演，一定要看，而且还要看完，90后的创...,39,76,123,2017-08-07 13:00,微博
2,3241971684,#西安身边事#【网友呼吁西安改回长安】西安古称长安，曾是古代第一大城市，与古罗马齐名！最近很...,32,226,284,2017-08-06 12:54,微博
3,3088945540,这个法官自己都不知道，他在中国有多红……👍#西安流年#L西安流年的秒拍视频（使用#秒拍#录制...,105,46,47,2017-08-07 22:29,微博
4,2277260272,#西安生活#喝完水之后，狗狗无比露出了灿烂的笑容。看到感觉可以开心一整天！L秒拍视频,44,34,115,2017-08-07 07:52,微博
...,...,...,...,...,...,...,...
49211,2860954042,#晚间路况#长江大桥南向北、盐仓桥广场立交南向北、新庄立交往玄武湖隧道、西安门隧道-通济门隧...,293,79,4,2019-03-09 20:56,微博
49212,2719969734,西安美食地图这是一份完全意义上的西安美食攻略，纯美食！按照钟楼，东西南北四条大街划分，无论你...,8,4,2,2019-03-10 20:01,微博
49213,5409023948,#西安生活#【回民街美食——红红酸菜炒米】西安不仅有凉皮冰封肉夹馍，回民街这家“红红酸菜炒米...,115,41,15,2019-03-05 21:57,微博
49214,1277255972,#迎接春天#【寻春记•寒窑】心有芬芳，春天就好。不知不觉间，春天已有了自己的模样，万物复苏，...,26,29,14,2019-03-06 16:41,微博


In [198]:
# data_preprocess(weibo, '微博')
# weibo['分词'] = weibo['内容'].apply(chinese_word_cut)

In [199]:
zhihu = pd.read_csv("resource/西安回答详情.csv")
zhihu.rename(columns={'answer_content':'内容', 'publish_time':'发布时间', 'voters_num':'点赞数', 'comments_num':'评论数', 'author_url':'user_tag'}, inplace = True)
zhihu = zhihu[['内容', '发布时间', '点赞数', '评论数', 'user_tag']]
zhihu['发布时间'] = zhihu['发布时间'].str.replace('发布于', '').str.strip()
zhihu['点赞数'] = zhihu['点赞数'].str.replace('赞同', '').str.replace(' ', '').str.strip()
zhihu['评论数'] = zhihu['评论数'].str.replace('添加评论', '').str.replace('条评论', '').str.strip()
zhihu['平台'] = '知乎'
display(zhihu)
# print(zhihu.columns.to_list())
# data_preprocess(zhihu, '知乎')

,内容,发布时间,点赞数,评论数,user_tag,平台
0,这次疫情，因为新冠没死一个。但因为防疫防控却死了好几个。我大西安，官员的能力如何。能看出来吧...,2022-01-27 17:48,​6343,​568,https://www.zhihu.com/people/zhao-shi-66-8,知乎
1,说一个最近发生的事吧，西安解封应该是年前一周，到现在已经半个多月过去了。我们小区比较大，平时...,2022-02-08 19:29,​1952,​416,https://www.zhihu.com/people/chang-yu-50-15,知乎
2,首先道出一个悲伤的现实：\n\n那就是，虽然你看似可以去任何地方，但最终你只能停留在你买房的...,2022-01-30 17:38,​873,​65,https://www.zhihu.com/people/TheDiamondSea,知乎
3,说实话，要不是西安爆发第二轮疫情，很多人真的以为西安是繁荣昌盛。但实际上真的是千疮百孔。\n...,2022-01-08 10:53,​7704,"​2,202",https://www.zhihu.com/people/chui-niu-da-wang-63,知乎
4,本人券商投行。2010年曾和同事一起调研过西安的军工企业。印象最深的是，我是最晚到的一批。然...,2022-01-11 02:41,​1223,​445,https://www.zhihu.com/people/chu-chu-chu-chu-qi,知乎
...,...,...,...,...,...,...
20207,高新、软件新城、沣东的交界\n\n,2021-05-06 10:48,​,​,https://www.zhihu.com/people/li-xiang-yu-54-3,知乎
20208,西安的环路真的是，南三环平均只敢开60，一路红绿灯加大井盖泥土的，一堵就是四五百米，东西三环...,2021-04-15 10:23,​3,​,https://www.zhihu.com/people/jiu-mei-zai-na-li,知乎
20209,软件新城上班，每天经过西三环，鱼化寨。附近交通比较乱，三环边上的绿植总是一层灰，没有啥自行车...,2021-03-06 07:25,​3,​,https://www.zhihu.com/people/science-2,知乎
20210,比东三环差多了，都什么年代了还在居民集中区建330kv超高压线，拿着十年年的环评报告施工，十...,2021-11-24 17:55,​2,​1,https://www.zhihu.com/people/simplehappy-57,知乎


In [200]:
douban = pd.read_csv("resource/豆瓣日记内容.csv")
douban['内容'] = douban['标题'] + douban['内容']
douban.rename(columns={'作者主页':'user_tag', '喜欢数':'点赞数'}, inplace = True)
douban = douban[['内容', '发布时间', '点赞数', '收藏数', '转发数', 'user_tag']]
# display(douban)
douban['平台'] = '豆瓣'
print(douban.info())
# data_preprocess(douban, '豆瓣')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1623 entries, 0 to 1622
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   内容        1357 non-null   object
 1   发布时间      1622 non-null   object
 2   点赞数       1623 non-null   int64 
 3   收藏数       1623 non-null   int64 
 4   转发数       1623 non-null   int64 
 5   user_tag  1622 non-null   object
 6   平台        1623 non-null   object
dtypes: int64(3), object(4)
memory usage: 88.9+ KB
None


### 数据集合处理

In [203]:
data = pd.concat([douban, weibo, zhihu], axis=0)
data = data.applymap(clean_data)
data[['内容']] = data[['内容']].applymap(clean_content)
data[['发布时间']] = data[['发布时间']].applymap(clean_time_data)
data[['点赞数', '收藏数', '转发数', '评论数']] = data[['点赞数', '收藏数', '转发数', '评论数']].fillna('0').applymap(clean_number).astype('int')
data.reset_index(drop=True, inplace=True)
display(data)
data.to_pickle('data/西安城市形象原始数据.pkl')

,内容,发布时间,点赞数,收藏数,转发数,user_tag,平台,评论数
0,西安旅游攻略吧，小伙伴年底要来西安玩一周！😂😂😂太高兴了。但是你要问一个西安人，西安有啥玩的...,2021-11-21 08:14:20,64,144,18,https://www.douban.com/people/Ch95134/,豆瓣,0
1,西安回民街到底该不该去，是不是专门坑游客的地方五一到了，很多人来西安玩会有个疑惑回民街到底该...,2021-04-24 15:58:39,186,442,38,https://www.douban.com/people/nnnpppccc/,豆瓣,0
2,西安怕是不能再来了~我现在对我的西安朋友充满了怨愤。因为他们要是早一点告诉我西安这么好吃，我...,2019-09-16 18:42:50,758,2782,613,https://www.douban.com/people/ChannaDisco/,豆瓣,0
3,西安是个很好玩的城市，景点和骗子都很好玩。首先想说的是，作为十三朝古都，西安是一个非常好的城...,2017-01-05 13:16:47,10789,11026,2816,https://www.douban.com/people/xiangyi816/,豆瓣,0
4,博物馆游记｜西安（三）第四站·秦始皇帝陵博物院（秦始皇兵马俑博物馆）秦始皇兵马俑，号称“世界...,2021-06-26 20:25:30,10,6,1,https://www.douban.com/people/qubing/,豆瓣,0
...,...,...,...,...,...,...,...,...
20207,高新、软件新城、沣东的交界,2021-05-06 10:48,0,0,0,https://www.zhihu.com/people/li-xiang-yu-54-3,知乎,0
20208,西安的环路真的是，南三环平均只敢开60，一路红绿灯加大井盖泥土的，一堵就是四五百米，东西三环...,2021-04-15 10:23,3,0,0,https://www.zhihu.com/people/jiu-mei-zai-na-li,知乎,0
20209,软件新城上班，每天经过西三环，鱼化寨。附近交通比较乱，三环边上的绿植总是一层灰，没有啥自行车...,2021-03-06 07:25,3,0,0,https://www.zhihu.com/people/science-2,知乎,0
20210,比东三环差多了，都什么年代了还在居民集中区建330kv超高压线，拿着十年年的环评报告施工，十...,2021-11-24 17:55,2,0,0,https://www.zhihu.com/people/simplehappy-57,知乎,1


### 词典数据处理

In [219]:
topic_data = pd.read_excel('resource/编码框架及关键词列表.xlsx')
narration_data = pd.read_excel('resource/编码框架及关键词列表.xlsx', sheet_name=1)
topic_data['编码类型'] = '议题编码'
narration_data['编码类型'] = '叙事编码'
topic_data.columns = narration_data.columns = ['一级框架类型','二级框架类型','框架说明', '框架关键词', '编号', '类型']
dictionary_data = pd.concat([topic_data[1:], narration_data[1:]], axis=0)
dictionary_data.to_pickle('data/西安项目编码表.pkl')
display(dictionary_data)


,一级框架类型,二级框架类型,框架说明,框架关键词,编号,类型
1,宏观议题,政治环境,关注西安市战略公布、政策制定、政府职能建设及政治会议、政治学习相关进程,办事处、宝鸡市、标志性、代表性、对外开放、发布会、负责人、共产党、管委会、国际化、国家级、国...,1,议题编码
2,宏观议题,经济发展,关注西安宏观经济情况，经济发展战略，现存经济现象、问题、产业发展相关信息,gdp、房地产、开发区、开发商、企事业、人民币、有限公司、总经理、cbd、产业链、第二产业、...,2,议题编码
3,宏观议题,生态建设,关注西安生态建设策略、工程，环境法规出台，城市绿化建设情况，环保普及教育相关进程,银杏树、大自然、红豆杉,3,议题编码
4,宏观议题,文化风俗,关注西安文化遗产、历史古迹、历史脉络、地方风俗、地理风貌、城市风光等内容,阿房宫、白鹿原、兵马俑、博物馆、博物院、车水马龙、城隍庙、大都市、大街小巷、大西北、风土人情...,4,议题编码
5,宏观议题,重大事件,关注西安市重大项目、活动、赛事等事件发生的各个阶段、进程，以及对活动的记载,世博园、意大利、交流会、洽谈会、奥运会、闭幕式、电影节、冬奥会、开幕式、马拉松、全运会、挑战...,5,议题编码
6,中观议题,司法治安,关注西安市相关部门对突发重大事件的解决方式和司法程序执行情况,110、当事人、派出所、被告人、公安部、公安局、检察官、检察院、目击者、人民法院、嫌疑人、有...,6,议题编码
7,中观议题,医疗卫生,关注西安市医疗卫生体系建设、医疗政策法规颁布、医疗信息公开、医疗知识普及进程及疫情相关信息,高风险、流行病、传染源、心脏病、hpv、出血热、感染者、后遗症、救护车、葡萄糖、普通型、软组...,7,议题编码
8,中观议题,基础设施,关注西安市道路交通、城市建设、能源供给、给水排水、通信等基础设施建设情况,出租车、地铁站、高峰期、高速公路、火车票、火车站、建筑物、客流量、客运站、人行道、主干道、高...,8,议题编码
9,中观议题,科教文艺,关注西安市科技发展、人才引进、教育资源、人才培养、文艺事业发展相关情况,211、985、本科生、分数线、机器人、计算机、教育局、美术馆、师范大学、实验室、体育场、图...,9,议题编码
10,微观议题,休闲娱乐,关注西安居民、游客及其能够参与的游览观光活动、社区互动活动、文艺体育活动，能够观看的文化汇演...,app、biangbiang、ktv、skp、爱好者、冰激凌、冰淇淋、不倒翁、步行街、吃吃喝...,10,议题编码


## playgroud